# Problem set #7
### by Alexander Tyan, March 5, 2018

## 1). Classier horse race (10 points). For this problem, you will use the 397 observations from the Auto.csv dataset.1 This dataset includes 397 observations on miles per gallon (mpg), number of cylinders (cylinders), engine displacement (displacement), horsepower (horsepower), vehicle weight (weight), acceleration (acceleration), vehicle year (year), vehicle origin (origin), and vehicle name (name). We will study the factors that make miles per gallon high or low. Create a binary variable mpg high that equals 1 if mpg high >= median(mpg high) and equals either 0 if mpg high< median(mpg high).

In [471]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import svm

In [472]:
df = pd.read_csv("Auto.csv", na_values=["?"])
mpg_median = df["mpg"].median()
df["mpg_high"] = (df["mpg"] >= mpg_median).astype(int)
df["constant"] = 1
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 396
Data columns (total 11 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null int64
acceleration    392 non-null float64
year            392 non-null int64
origin          392 non-null int64
name            392 non-null object
mpg_high        392 non-null int64
constant        392 non-null int64
dtypes: float64(4), int64(6), object(1)
memory usage: 36.8+ KB


### a). Use sklearn.linear model.LogisticRegression to t a logistic model of mpg high on features number of cylinders (cyl), engine displacement (dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehicle year (yr), vehicle origin (orgn). Make sure to include a constant term.

In [473]:
X_values = df[['constant', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
X_values_no_const = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y_values = df['mpg_high'].values

In [474]:
k = 4
kf_log = KFold(n_splits=k, shuffle=True, random_state=15)
kf_log.get_n_splits(X_values)

MSE_vec_kf_log = np.zeros(k)
error_vec_cat_0 = np.zeros(k)
error_vec_cat_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_log.split(X_values):
    X_train, X_test = X_values[train_index], X_values[test_index]
    y_train, y_test = y_values[train_index], y_values[test_index]
    LogReg = linear_model.LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf_log[k_ind] = ((y_test - y_pred) ** 2).mean()
    success_stats = precision_recall_fscore_support(y_test, y_pred)
    cat_0_precision = success_stats[0][0]
    cat_0_error_rate = 1 - cat_0_precision
    cat_1_precision = success_stats[0][1]
    cat_1_error_rate = 1 - cat_1_precision
    error_vec_cat_0[k_ind] = cat_0_error_rate
    error_vec_cat_1[k_ind] = cat_1_error_rate
    k_ind += 1

### Report the MSE of the model as the average MSE across the k = 4 test sets, and report the error rates for each category of mpg high as the average error rate for that category across the k = 4 test sets.

In [475]:
MSE_kf = MSE_vec_kf_log.mean()
MSE_kf_std = MSE_vec_kf_log.std()
error_cat_0 = error_vec_cat_0.mean()
error_cat_0_std = error_vec_cat_0.std()
error_cat_1 = error_vec_cat_1.mean()
error_cat_1_std = error_vec_cat_1.std()
print('Mean MSE (Logistic Regression) =', MSE_kf)
print('Mean SE of MSE (Logistic Regression) =', MSE_kf_std)
print('Mean Error Rate for Category 0 (Logistic Regression) =', error_cat_0)
print('Mean SE of Error for Category 0 (Logistic Regression) =', error_cat_0_std)
print('Mean Error Rate for Category 1 (Logistic Regression) =', error_cat_1)
print('Mean SE of Error for Category 1 (Logistic Regression) =', error_cat_1_std)

Mean MSE (Logistic Regression) = 0.107142857143
Mean SE of MSE (Logistic Regression) = 0.015306122449
Mean Error Rate for Category 0 (Logistic Regression) = 0.0947067070586
Mean SE of Error for Category 0 (Logistic Regression) = 0.0440594622751
Mean Error Rate for Category 1 (Logistic Regression) = 0.118226649127
Mean SE of Error for Category 1 (Logistic Regression) = 0.0236516096988


### b). Use sklearn.ensemble.RandomForestClassifier to t a random forest model of mpg high on max features=2 out of the seven possible features used in part (a). Set n estimators=20, set bootstrap=True, set oob score=True, and set random state=25.


In [476]:
rand_forest_model = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                           oob_score=True, random_state=25)
rand_forest_model.fit(X_values_no_const, y_values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

### Report the MSE of the random forest model as the MSE from the .oob prediction object, and report the error rates for each category of mpg high from the .oob prediction object.

In [477]:
y_pred_forest = rand_forest_model.oob_decision_function_[:,1] - rand_forest_model.oob_decision_function_[:,0]
y_pred_forest[y_pred_forest > 0] = np.int(1)
y_pred_forest[y_pred_forest <= 0] = np.int(0)

MSE_forest = mean_squared_error(y_values, y_pred_forest)
success_stats = precision_recall_fscore_support(y_values, y_pred_forest)
cat_0_precision = success_stats[0][0]
cat_0_error_rate = 1 - cat_0_precision
cat_1_precision = success_stats[0][1]
cat_1_error_rate = 1 - cat_1_precision


print('MSE (Random Forest Classifier) =', MSE_forest)
print('Error Rate for Category 0 (Random Forest Classifier) =', cat_0_error_rate)
print('Error Rate for Category 1 (Random Forest Classifier) =', cat_1_error_rate)

MSE (Random Forest Classifier) = 0.0663265306122
Error Rate for Category 0 (Random Forest Classifier) = 0.0618556701031
Error Rate for Category 1 (Random Forest Classifier) = 0.0707070707071


### c). Use sklearn.svm.SVC to t a support vector machines model of mpg high with a Gaussian radial basis function kernel kernel='rbf' on the seven features used in part (a). Set the penalty parameter to C=1 and set gamma=0.2. Fit the model using k-fold cross validation with k = 4 folds exactly as in part (a).

In [478]:
k = 4
kf_svc = KFold(n_splits=k, shuffle=True, random_state=15)
kf_svc.get_n_splits(X_values_no_const)

MSE_vec_kf_svc = np.zeros(k)
error_vec_cat_0 = np.zeros(k)
error_vec_cat_1 = np.zeros(k)

k_ind = int(0)

for train_index, test_index in kf_svc.split(X_values_no_const):
    X_train, X_test = X_values_no_const[train_index], X_values_no_const[test_index]
    y_train, y_test = y_values[train_index], y_values[test_index]
    
    svc_rbf = svm.SVC(kernel='rbf', gamma=0.2, C=1)
    svc_rbf.fit(X_train, y_train)
    y_pred_rbf = svc_rbf.predict(X_test)
    
    success_stats = precision_recall_fscore_support(y_test, y_pred_rbf)
    cat_0_precision = success_stats[0][0]
    cat_0_error_rate = 1 - cat_0_precision
    cat_1_precision = success_stats[0][1]
    cat_1_error_rate = 1 - cat_1_precision
    
    cat_0_num_cases = confusion_matrix(y_test, y_pred_rbf)[0,0] + confusion_matrix(y_test, y_pred_rbf)[1,0]
    cat_1_num_cases = confusion_matrix(y_test, y_pred_rbf)[1,1] + confusion_matrix(y_test, y_pred_rbf)[0,1]
   
    error_vec_cat_0[k_ind] = cat_0_error_rate
    
    # Avoid no predicted samples case in in the second loop:
    if k_ind != 1:
        error_vec_cat_1[k_ind] = cat_1_error_rate
    
    MSE_rbf = mean_squared_error(y_test, y_pred_rbf)
    MSE_vec_kf_svc[k_ind] = MSE_rbf
    
    k_ind += 1
    
# Avoid no predicted samples case in in the second loop:
error_vec_cat_1 = np.delete(error_vec_cat_1, 1)

/Users/alexander/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Report the MSE of the model as the average MSE across the k = 4 test sets, and report the error rates for each category of mpg high as the average error rate for that category across the k = 4 test sets.

In [479]:
mean_SVC_MSE = MSE_vec_kf_svc.mean()
SVC_MSE_std = MSE_vec_kf_svc.std()
error_cat_0 = error_vec_cat_0.mean()
error_cat_0_std = error_vec_cat_0.std()
error_cat_1 = error_vec_cat_1.mean()
error_cat_1_std = error_vec_cat_1.std()
print("Mean MSE (SVC Model) = ", mean_SVC_MSE)
print('Mean SE of MSE (SVC Model) =', SVC_MSE_std)
print('Mean Error Rate for Category 0 (SVC Model) =', error_cat_0)
print('Mean SE of Error for Category 0 (SVC Model) =', error_cat_0_std)
print('Mean Error Rate for Category 1 (SVC Model) =', error_cat_1)
print('Mean SE of Error for Category 1 (SVC Model) =', error_cat_1_std)

Mean MSE (SVC Model) =  0.507653061224
Mean SE of MSE (SVC Model) = 0.0348846794163
Mean Error Rate for Category 0 (SVC Model) = 0.381194110709
Mean SE of Error for Category 0 (SVC Model) = 0.221165280503
Mean Error Rate for Category 1 (SVC Model) = 0.184027777778
Mean SE of Error for Category 1 (SVC Model) = 0.260254579187


### d). Which of the above three models do you think is the best predictor of mpg high? Why?

Judging by the MSE values, Random Forest Classifier is the best, with the MSE value of 0.0663265306122. This indicates low average errors between what the model expects in a test set and what value test set actually has and is thus an indicator of predictive power. Suprisingly, the SVC model is the worst, with the MSE of 0.507653061224 (i.e. high predictive error rates). I would have thought that the SVC would be better at binary classification than the RFC. 